In [1]:
import cv2
import imutils
import time

greenLower = (42,85,85)#(36,175,160)
#bouncing TENIS ball=>   videos:(42,85,85)      office: (22,51,51)
#bouncing SMALL ball=>   videos:(42,85,85)
greenUpper = (212,254,251)#(75,238,200)#(76,238,200)
#bouncing TENIS ball=>   videos:(212,254,251)    office: (212,254,251)
#bouncing SMALL ball=>   videos:(85,204,165) 

#to crop
cropping=True
if cropping: #for bouncing ball: 0 1200 750 1200
    lower= 0
    upper= 1200 
    left= 750
    right= 1200

#vs = cv2.VideoCapture(0)
vs = cv2.VideoCapture("MVI_2049.MOV") #0 if webcam,video name else

time.sleep(2.0)

while True:
    _, frame = vs.read()

    if frame is None:
        break
    if cropping:
        frame = frame[lower:upper,left:right]  
    
    blurred = cv2.GaussianBlur(frame, (17, 17), 0)
    width, height = frame.shape[:2]
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, greenLower, greenUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None

    if len(cnts) > 0:
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

        # To see the centroid clearly
        if radius > 10:
            cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 5)
            cv2.imwrite("circled_frame.png", cv2.resize(frame, (int(height / 2), int(width / 2))))
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
    
#    if cropping:
#        cropped = frame[lower:upper,left:right]     
#        cv2.imshow("cropped",cropped)   
#    else:
#        cv2.imshow("Frame", frame)
    resized = cv2.resize(frame, (200,500))
    cv2.imshow("Frame", resized)
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

vs.release()
cv2.destroyAllWindows()